In [ ]:
!nvidia-smi

Mon Aug 24 08:08:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!python -m pip install git+https://github.com/ShivamShrirao/dnn_from_scratch.git

  Cloning https://github.com/ShivamShrirao/dnn_from_scratch.git to /tmp/pip-req-build-9qf32oxg
  Running command git clone -q https://github.com/ShivamShrirao/dnn_from_scratch.git /tmp/pip-req-build-9qf32oxg
  Created wheel for dnn-from-scratch: filename=dnn_from_scratch-0.1.dev1-cp36-none-any.whl size=39847 sha256=01a1375b9c06e7edc746df0c24561f3f86cf21b34c77b1cf008a835fbfa4cf4d
  Stored in directory: /tmp/pip-ephem-wheel-cache-3dwvotrk/wheels/33/77/6c/1140f6295368e7045f3b49fdda98e7f07b2513b8b55093ca7b
Successfully built dnn-from-scratch


In [ ]:
!git clone https://github.com/ShivamShrirao/deep_Q_learning_from_scratch.git

Cloning into 'deep_Q_learning_from_scratch'...
remote: Enumerating objects: 271, done.
remote: Counting objects: 100% (271/271), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 271 (delta 173), reused 186 (delta 88), pack-reused 0
Receiving objects: 100% (271/271), 216.32 KiB | 1.21 MiB/s, done.
Resolving deltas: 100% (173/173), done.


In [ ]:
%cd deep_Q_learning_from_scratch

/content/deep_Q_learning_from_scratch


In [ ]:
!git pull origin master

From https://github.com/ShivamShrirao/deep_Q_learning_from_scratch
 * branch            master     -> FETCH_HEAD
Already up to date.


In [ ]:
import gym
import time
import cv2
import numpy as np
from collections import deque

from settings import *
from agent import *
from experience import *

In [ ]:
from nnet_gpu.network import Sequential
from nnet_gpu.layers import Conv2D,Flatten,Dense,Dropout
from nnet_gpu import optimizers
from nnet_gpu import functions

In [ ]:
HEIGHT = 80
WIDTH = 72

In [ ]:
def get_model(input_shape=(HEIGHT,WIDTH,NFRAMES), no_of_actions=3):
    model=Sequential()
    model.add(Conv2D(num_kernels=32, kernel_size=3, stride=(2, 2), activation=functions.relu, input_shape=input_shape))
    model.add(Dropout(0.1))
    model.add(Conv2D(num_kernels=64, kernel_size=3, stride=(2, 2), activation=functions.relu))
    model.add(Dropout(0.2))
    model.add(Conv2D(num_kernels=128, kernel_size=3, stride=(2, 2), activation=functions.relu))
    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(512, activation=functions.relu))
    model.add(Dense(no_of_actions, activation=functions.echo))

    model.compile(optimizer=optimizers.adam, loss=functions.mean_squared_error, learning_rate=0.0001)
    return model

def preproc_obsv(obsv):
    obsv = cv2.cvtColor(obsv, cv2.COLOR_RGB2GRAY)
    obsv = obsv[34:194:2,8:152:2]
    return obsv

In [ ]:
agt = DQN_Agent(actions=[0,2,3], epsilon=1, min_epsilon=0.1, eps_decay=1e-6, target_update_thresh=3000)
agt.model = get_model(input_shape=(HEIGHT,WIDTH,NFRAMES), no_of_actions=len(agt.actions))
agt.target = get_model(input_shape=(HEIGHT,WIDTH,NFRAMES), no_of_actions=len(agt.actions))
agt.model.load_weights("/content/drive/My Drive/breakout.w8s")
agt.update_target()
agt.model.summary()

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
- InputLayer(InputLayer)  (None, 80, 72, 4)          echo             0
__________________________________________________________________________________________
0 Conv2D(Conv2D)          (None, 40, 36, 32)         relu             1184
__________________________________________________________________________________________
1 Dropout(Dropout)        (None, 40, 36, 32)         echo             0
__________________________________________________________________________________________
2 Conv2D(Conv2D)          (None, 20, 18, 64)         relu             18496
__________________________________________________________________________________________
3 Dropout(Dropout)        (None, 20, 18, 64)         echo             0
__________________________________________________________________________________________
4 Conv2D(

In [ ]:
D_exp = ReplayMemory(capacity=1_000_000, nlap=1, height=HEIGHT, width=WIDTH)

In [ ]:
D_exp.current_state.nbytes/1024/1024/1024

5.364418029785156

In [ ]:
env = gym.make('Breakout-v4')       # v4 to ensure ball is fired when required

In [ ]:
total_frames = 0
scores = deque([], maxlen=100)

In [ ]:
D_exp.len

599364

In [ ]:
for i_episode in range(3000):
    obinit = env.reset()
    if not i_episode:
        observation = obinit
        state = preproc_obsv(observation)
        state_que = deque([], maxlen=NFRAMES)
        for i in range(NFRAMES):
            state_que.append(state)
    ep_score = 0
    lives = 5
    fire = True
    start = time.time()
    for t in range(10_000):
        s_s = time.time()
#         env.render()
        state = preproc_obsv(observation)
        state_que.append(state)
        if fire:
            action = 1
            fire = False
        else:
            action = agt.get_action(state_que)
        next_observation, reward, done, info = env.step(action)
        ep_score += reward
        if lives != info['ale.lives']:
            lives = info['ale.lives']
            reward = -2
            fire = True
        reward = np.clip(reward, -2, 2)
        if action==1:
            action = 0
        D_exp.store_transition(state, agt.actions.index(action), reward, done)
        observation = next_observation

        if (D_exp.len-D_exp.idx_len) > BATCH_SIZE:
            grads = agt.train(D_exp, BATCH_SIZE)
        print('\r', t, action, ep_score, end='  ')
        if done:
            break
    total_frames += (t+1)
    scores.append(ep_score)
    print(f"\rEpisode {i_episode+1} finished after {t+1} timesteps, Score: {ep_score}, Avg: {sum(scores)/len(scores):.2f}, Epsilon: {agt.epsilon:.6f}, Time: {time.time()-start:.2f}, Frames: {total_frames:,}")
    if not i_episode%8:
        agt.model.save_weights("/content/drive/My Drive/breakout.w8s")
        if total_frames > 1_000_000:
            agt.continue_decay = True
env.close()

In [ ]:
agt.epsilon = 0.11
agt.min_epsilon = 0.1
agt.eps_decay = 1e-6
agt.target_update_thresh = 3000
agt.continue_decay = False

In [ ]:
from gym import wrappers

In [ ]:
wrapped_env = wrappers.Monitor(env.env, '/content/videos/' + str(time.time()) + '/')
for i_episode in range(1):
    obinit = wrapped_env.reset()
    if not i_episode:
        observation = obinit
        state = preproc_obsv(observation)
        state_que = deque([], maxlen=NFRAMES)
        for i in range(NFRAMES):
            state_que.append(state)
    ep_score = 0
    lives = 5
    fire = True
    preds = []
    reward_history = []
    start = time.time()
    t=-1
    while 1:
        t+=1
        s_s = time.time()
#         wrapped_env.render()
        state = preproc_obsv(observation)
        state_que.append(state)
        if fire:
            action = 1
            fire = False
        else:
            out = agt.predict(state_que)
            pidx = cp.argmax(out[0]).item()
            preds.append(out[0][pidx].item())
            action = agt.actions[pidx]
        next_observation, reward, done, info = wrapped_env.step(action)
        if lives != info['ale.lives']:
            lives = info['ale.lives']
            reward = -1
            fire = True
        ep_score += reward
        reward_history.append(reward)
        if action==1:
            action = 0
        observation = next_observation
        print('\r', t, ep_score, end='  ')
        if done:
            break
    print(f"\rEpisode {i_episode+1} finished after {t+1} timesteps, Score: {ep_score}, Epsilon: {agt.epsilon:.6f}, Time: {time.time()-start:.2f}")
wrapped_env.close()

In [ ]:
!rm /content/videos/* -rf